<a href="https://colab.research.google.com/github/sljm12/machine_learning_notebooks/blob/master/Bert_Next_Sentence_for_breaking_up_paragraphs_and_summarizing_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Testing the idea of usng Bert Next Sentence to break up news/speechs into different segments.

Then afterwhich running a summarizer on the segments to produce a long summary.

In [139]:
!pip -q install transformers

In [140]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForNextSentencePrediction

In [141]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForNextSentencePrediction.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForNextSentencePrediction: ['mlm___cls']
- This IS expected if you are initializing TFBertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


# Preprocess the text

In [142]:
!pip -q install newspaper3k

In [143]:
from newspaper import Article, ArticleException

In [144]:
url = "https://www.asiaone.com/entertainment/you-wont-see-zoe-tay-new-show-while-and-its-because-her-sons" #@param ["https://www.straitstimes.com/singapore/the-news-in-5-minutes-6","https://www.pmo.gov.sg/Newsroom/National-Day-Rally-2019","https://www.channelnewsasia.com/news/asia/malaysia-anwar-ibrahim-strong-majority-form-new-government-13136908","https://www.bbc.com/news/world-asia-54174598","https://www.straitstimes.com/tech/ibm-to-train-800-mid-career-people-in-ai-cyber-security","https://www.asiaone.com/entertainment/darren-lim-turns-his-yacht-business-opportunity-takes-people-out-rides","https://www.asiaone.com/entertainment/you-wont-see-zoe-tay-new-show-while-and-its-because-her-sons"]
a=Article(url)
a.download()
a.parse()
text = a.text

In [145]:
filename = url.replace(":","").replace("/","_")

In [146]:
text

'For the rest of the year, local \'Ah Jie\' Zoe Tay will be playing one role only and it won\'t be for telly.\n\nIn an interview with Shin Min Daily News, the veteran actress revealed that she\'ll be stepping back from acting so she can help her three sons prepare for their examinations — especially her second child Ashton who entered secondary school this year.\n\nShe also has Brayden, 15, and Nathan, nine, with her husband Philip Chionh.\n\nThe earliest we\'ll probably see her on our television screens again will be next year. Not too long a wait for the eager fans if you ask us.\n\nIn the meantime, apart from prepping her children for their exams, the 52-year-old will be using the downtime to focus on her fitness. She said: "After sending my children to school, I\'ll occasionally go to MacRitchie Reservoir for a walk."\n\n"The old [Mediacorp] station was located opposite the reservoir but for over 30 years, I rarely explored the area, though we would shoot there occasionally. Recent

We want to extract only the sentences. Minus the headers. We split by \n\n then we only choose those that ends with a full stop

In [147]:
text = [t for t in text.split("\n\n") if t.endswith('.')]

In [148]:
text

["For the rest of the year, local 'Ah Jie' Zoe Tay will be playing one role only and it won't be for telly.",
 "In an interview with Shin Min Daily News, the veteran actress revealed that she'll be stepping back from acting so she can help her three sons prepare for their examinations — especially her second child Ashton who entered secondary school this year.",
 'She also has Brayden, 15, and Nathan, nine, with her husband Philip Chionh.',
 "The earliest we'll probably see her on our television screens again will be next year. Not too long a wait for the eager fans if you ask us.",
 '"The old [Mediacorp] station was located opposite the reservoir but for over 30 years, I rarely explored the area, though we would shoot there occasionally. Recently, my friend brought me to this paradise (MacRitchie Reservoir) and I love the natural scenery. I can breathe the fresh air and train my body, it kills two birds with one stone," she added.',
 "While she's not taking on acting roles, she will s

Break down into sentences 

In [149]:
results = []
for i in text:
  c = [t.strip()+"." for t in i.split(".") if len(t) != 0]
  results = results + c

In [150]:
text = results

# Running Bert NextSentence to detect paragraphs

In [151]:
def compute(text1, text2):
  encoding = tokenizer(text1, text2, return_tensors='tf')
  logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]
  return tf.nn.softmax(logits[0])

Loop through the sentences adding sentences to temp group when the sentences are the next sentences.

When they are not the next sentence we append the temp_group to groups and reinit the temp_group

In [152]:
groups = []
groups_counter=0
temp_group = []
for i in range(len(text)-1):
  print("Group "+str(i))
  print(text[i])
  print(text[i+1])
  
  sm = compute(text[i], text[i+1])
  print(sm[0], sm[1])

  if len(temp_group) == 0:
    temp_group = [text[i]]

  if sm[0] > 0.9995 : #Threshold for similarity we set at this
      temp_group = temp_group + [text[i+1]]
  else:
    groups.append(temp_group)
    temp_group = [text[i+1]]

Group 0
For the rest of the year, local 'Ah Jie' Zoe Tay will be playing one role only and it won't be for telly.
In an interview with Shin Min Daily News, the veteran actress revealed that she'll be stepping back from acting so she can help her three sons prepare for their examinations — especially her second child Ashton who entered secondary school this year.
tf.Tensor(0.9999882, shape=(), dtype=float32) tf.Tensor(1.1760931e-05, shape=(), dtype=float32)
Group 1
In an interview with Shin Min Daily News, the veteran actress revealed that she'll be stepping back from acting so she can help her three sons prepare for their examinations — especially her second child Ashton who entered secondary school this year.
She also has Brayden, 15, and Nathan, nine, with her husband Philip Chionh.
tf.Tensor(0.99998057, shape=(), dtype=float32) tf.Tensor(1.9485651e-05, shape=(), dtype=float32)
Group 2
She also has Brayden, 15, and Nathan, nine, with her husband Philip Chionh.
The earliest we'll prob

In [153]:
groups

[["For the rest of the year, local 'Ah Jie' Zoe Tay will be playing one role only and it won't be for telly.",
  "In an interview with Shin Min Daily News, the veteran actress revealed that she'll be stepping back from acting so she can help her three sons prepare for their examinations — especially her second child Ashton who entered secondary school this year.",
  'She also has Brayden, 15, and Nathan, nine, with her husband Philip Chionh.',
  "The earliest we'll probably see her on our television screens again will be next year.",
  'Not too long a wait for the eager fans if you ask us.'],
 ['"The old [Mediacorp] station was located opposite the reservoir but for over 30 years, I rarely explored the area, though we would shoot there occasionally.',
  'Recently, my friend brought me to this paradise (MacRitchie Reservoir) and I love the natural scenery.',
  'I can breathe the fresh air and train my body, it kills two birds with one stone," she added.'],
 ["While she's not taking on a

In [154]:
import json
from pathlib import Path

Path(filename+"_groups.json").write_text(json.dumps(groups))

1464

# Summarization

## T5 Summarizer

In [155]:
from transformers import pipeline
summarizer = pipeline("summarization")

In [156]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

def t5_summarise(text):
  model = T5ForConditionalGeneration.from_pretrained('t5-small')
  tokenizer = T5Tokenizer.from_pretrained('t5-small')
  device = torch.device('cpu')

  preprocess_text = text.strip().replace("\n","")
  t5_prepared_Text = "summarize: "+preprocess_text
  #print ("original text preprocessed: \n", preprocess_text)

  tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


  # summmarize 
  summary_ids = model.generate(tokenized_text,
                                      num_beams=4,
                                      no_repeat_ngram_size=2,
                                      min_length=30,
                                      max_length=200,
                                      early_stopping=False)

  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  #print ("\n\nSummarized text: \n",output)
  return output

In [157]:
' '.join(groups[0])

"For the rest of the year, local 'Ah Jie' Zoe Tay will be playing one role only and it won't be for telly. In an interview with Shin Min Daily News, the veteran actress revealed that she'll be stepping back from acting so she can help her three sons prepare for their examinations — especially her second child Ashton who entered secondary school this year. She also has Brayden, 15, and Nathan, nine, with her husband Philip Chionh. The earliest we'll probably see her on our television screens again will be next year. Not too long a wait for the eager fans if you ask us."

In [158]:
summarised = []
for i in groups:
  joined = ' '.join(i)
  summarised.append(t5_summarise(joined))

In [159]:
len(summarised)

5

In [160]:
import pprint

In [161]:
pprint.pprint(summarised)

["'Ah Jie' Zoe Tay will be playing one role for the rest of the year. she'll "
 'be stepping back from acting so she can help her three sons prepare for '
 'their examinations, especially her second child Ashton who entered secondary '
 'school this year.',
 'the old [Mediacorp] station was located opposite the reservoir. recently, my '
 'friend brought me to this paradise (MacRitchie Reservoir)',
 "she'll perform a modern Indian dance routine with six beneficiaries of the "
 "annual fundraiser. she will also be part of president's star charity this "
 'year.',
 'the recent death of taiwanese celebrity Alien Huang has a ripple effect that '
 'can be acutely felt on our shores.',
 "Zoe didn't get a chance to work with him. she was able to find out if he was "
 'working with her.']


In [162]:
Path(filename+"_t5_sum.json").write_text(json.dumps(summarised))

754

In [163]:
summarised_text = ""
for i in summarised:
  summarised_text = summarised_text+ i+"\n\n"

In [164]:
print(summarised_text)

'Ah Jie' Zoe Tay will be playing one role for the rest of the year. she'll be stepping back from acting so she can help her three sons prepare for their examinations, especially her second child Ashton who entered secondary school this year.

the old [Mediacorp] station was located opposite the reservoir. recently, my friend brought me to this paradise (MacRitchie Reservoir)

she'll perform a modern Indian dance routine with six beneficiaries of the annual fundraiser. she will also be part of president's star charity this year.

the recent death of taiwanese celebrity Alien Huang has a ripple effect that can be acutely felt on our shores.

Zoe didn't get a chance to work with him. she was able to find out if he was working with her.




In [165]:
Path(filename+"_t5_sum.txt").write_text(summarised_text)

744

## Default hugging face summarizer pipeline

In [166]:
sum1 = []

for i in groups:
  t = ' '.join(i)
  sum1.append(summarizer(t))

Your max_length is set to 142, but you input_length is only 129. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 80. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 52. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 33. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 18. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 18. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


In [167]:
sum1

[[{'summary_text': " Zoe Tay will be stepping back from acting so she can help her three sons prepare for their examinations . The earliest we'll probably see her on our television screens again will be next year . She also has Brayden, 15, and Nathan, nine, with her husband Philip Chionh ."}],
 [{'summary_text': ' MacRitchie Reservoir was located opposite the old Mediacorp station . "For over 30 years, I rarely explored the area, though we would shoot there occasionally," she said . "I can breathe the fresh air and train my body, it kills two birds with one stone," she added .'}],
 [{'summary_text': " Apart from her editorial shoots, she'll be part of this year's President's Star Charity . She'll perform a modern Indian dance routine with six beneficiaries of the annual fundraiser . While she won't be taking on acting roles, she will still be working for the charity ."}],
 [{'summary_text': " The recent death of Taiwanese celebrity Alien Huang has a ripple effect that can be acutely f

The number of groups.

In [168]:
len(sum1)

5

In [169]:
Path(filename+"_hugging_sum.json").write_text(json.dumps(sum1))

1483

In [170]:
summary = [i[0]["summary_text"].strip() for i in sum1]

Path(filename+"_hugging_sum.txt").write_text('\n\n'.join(summary))

1362

In [171]:
summary

["Zoe Tay will be stepping back from acting so she can help her three sons prepare for their examinations . The earliest we'll probably see her on our television screens again will be next year . She also has Brayden, 15, and Nathan, nine, with her husband Philip Chionh .",
 'MacRitchie Reservoir was located opposite the old Mediacorp station . "For over 30 years, I rarely explored the area, though we would shoot there occasionally," she said . "I can breathe the fresh air and train my body, it kills two birds with one stone," she added .',
 "Apart from her editorial shoots, she'll be part of this year's President's Star Charity . She'll perform a modern Indian dance routine with six beneficiaries of the annual fundraiser . While she won't be taking on acting roles, she will still be working for the charity .",
 "The recent death of Taiwanese celebrity Alien Huang has a ripple effect that can be acutely felt on our shores . He has a strong connection to many local celebrities . The dea